In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
def GetPlottingDirectory(plotFileName, plotType):
    plottingDirectory = mainCodeDirectory=os.path.join(mainDirectory,"Code","PLOTTING")
    
    specificPlottingDirectory = os.path.join(plottingDirectory, plotType)
    os.makedirs(specificPlottingDirectory, exist_ok=True)

    plottingFileName=os.path.join(specificPlottingDirectory, plotFileName)

    return plottingFileName

In [ ]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, DataManager_Class

In [ ]:
####################################
#LOADING CLASSES

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=2)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str, #just need this one for the data loading function
                                ModelData.Np_str, dataType="Figures", dataName="Figures", dtype="float32")

In [ ]:
#GETTING DATA
data=ModelData.OpenData()

In [ ]:
####################################
#SPECIFIC LIBRARIES

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
####################################
#FUNCTIONS

In [ ]:
main_data_location = '/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/CodeFiles/1_Initial_Figures/Figures/Campaign_Data/'

In [ ]:
def GetData1(var_name):
    land_data = data[var_name].isel(xh=land).data#.mean(dim=['xh', 'yh'])
    ocean_data = data[var_name].isel(xh=ocean).data#.mean(dim=['xh', 'yh'])
    
    land_data_mean = np.mean(land_data,axis=(1,2))
    ocean_data_mean = np.mean(ocean_data,axis=(1,2))
    
    return land_data_mean,ocean_data_mean

def GetData2(var_name):
    land_data = data[var_name].isel(xh=land,zh=0).data#.mean(dim=['xh', 'yh'])
    ocean_data = data[var_name].isel(xh=ocean,zh=0).data#.mean(dim=['xh', 'yh'])
    
    land_data_mean = np.mean(land_data,axis=(1,2))
    ocean_data_mean = np.mean(ocean_data,axis=(1,2))
    
    return land_data_mean,ocean_data_mean

def SaveLandOceanData(data_list, filename=main_data_location+f'land_ocean_data_{ModelData.res}_{ModelData.t_res}.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(data_list, f)
    print(f"Saved land and ocean data to '{filename}'")

def LoadLandOceanData(filename=main_data_location+f'land_ocean_data_{ModelData.res}_{ModelData.t_res}.pkl'):
    with open(filename, 'rb') as f:
        data_list = pickle.load(f)
    return data_list[0], data_list[1]

In [ ]:
####################################
#PLOTTING

In [ ]:
#SETTING UP SOME PLOT THINGS

#time limit
dt_mins = int(ModelData.t_res[0])
timesteps_per_hour=60/dt_mins
previous_hours=11
previous_times=-timesteps_per_hour*previous_hours

#defining land and ocean slices
fraction=1/4
land = slice(int(len(ModelData.xh)*fraction), int(len(ModelData.xh)))
ocean = slice(0, int(len(ModelData.xh)*fraction))

In [ ]:
#PLOTTING
n_bins=10 #number of plotting bins


# Create figure and axes
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
fig.suptitle('Surface Area Means')

# Surface Temperature Plot
ax = axes[0, 0]
var_name='tsk'
[land_data_mean,ocean_data_mean] = GetData1(var_name)
SaveLandOceanData([land_data_mean,ocean_data_mean]) #SAVING HERE (FOR USE IN Reanalysis_Diurnal_Cycle)
ax.plot(land_data_mean, color='brown', label='land')
ax.plot(ocean_data_mean, color='blue', label='ocean')
ax.set_title('Surface Temperature (K)')
ax.set_xlabel('Timestep')
ax.set_ylabel('Temp (K)')
ax.legend(loc='lower left')
ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=n_bins))  # Set Y-Ticks
ax.set_xlim(left=previous_times) #matching data diurnal cycle

# QV Flux Plot
ax = axes[0, 1]
var_name='qvflux'
[land_data_mean,ocean_data_mean] = GetData1(var_name)
ax.plot(land_data_mean, color='brown', label='land')
ax.plot(ocean_data_mean, color='blue', label='ocean')
ax.set_title('QV Flux (K)')
ax.set_xlabel('Timestep')
ax.set_ylabel(r'qvflux ($g\ g^{-1} m\ s^{-1}$)')
ax.legend(loc='lower left')
ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=n_bins))  # Set Y-Ticks
ax.set_xlim(left=previous_times) #matching data diurnal cycle

# Theta Flux Plot
ax = axes[1, 0]
var_name='thflux'
[land_data_mean,ocean_data_mean] = GetData1(var_name)
ax.plot(land_data_mean, color='brown', label='land')
ax.plot(ocean_data_mean, color='blue', label='ocean')
ax.set_title('Theta Flux (K)')
ax.set_xlabel('Timestep')
ax.set_ylabel(r'thflux ($K\ m\ s^{-1}$)')
ax.legend(loc='lower left')
ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=n_bins))  # Set Y-Ticks
ax.set_xlim(left=previous_times) #matching data diurnal cycle

# U Wind Component Plot
ax = axes[1, 1]
var_name='uinterp'
[land_data_mean,ocean_data_mean] = GetData2(var_name)
ax.plot(land_data_mean, color='brown', label='land')
ax.plot(ocean_data_mean, color='blue', label='ocean')
ax.set_title(r'U ($m\ s^{-1}$)')
ax.set_xlabel('Timestep')
ax.set_ylabel(r'u ($m\ s^{-1}$)')
ax.legend(loc='lower left')
ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=n_bins))  # Set Y-Ticks
ax.set_xlim(left=previous_times) #matching data diurnal cycle